## 2.此程式:
先讀取飯店名字的.npy檔案，再去爬取所有這個hotel的資訊並存成csv檔

## Package loading

In [1]:
from selenium import webdriver #The Selenium package we'll need
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
import csv #This package lets us save data to a csv file
import re
import time #This package lets us pause execution for a bit
import pandas as pd
import numpy as np
from tqdm import tqdm
import math
import random
# 靜態爬蟲
from bs4 import BeautifulSoup
import requests
from pathlib import Path
import json


In [2]:
#讀取
hotel_data_read=np.load('/home/hao/Code/variable/hotel_data_la_store.npy', allow_pickle=True)
hotel_data=hotel_data_read.tolist()

hotels_list = []
for h in hotel_data:
    hotels_list.append('https://www.tripadvisor.com'+h['Url'])

len(hotels_list)

502

## Web scraping

In [3]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-images')
options.add_argument('--disable-gpu')
# options.add_argument("--disable-javascript")
# 無痕模式
options.add_argument('--incognito')

In [4]:
# driver setting
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
driver.maximize_window()
# driver.implicitly_wait(10)

In [5]:
# 獲得 geoId, locationId
def get_ids_from_hotel_url(url):

  url = url.split('-')

  geo = url[1]

  loc = url[2]

  return (int(geo[1:]), int(loc[1:]))

In [6]:
def get_hotel_info(hotel_url):
    row_data = []
    driver.get(hotel_url)
    time.sleep(2)
    
    # get locID, geoID
    geo, loc = get_ids_from_hotel_url(hotel_url)

    # Path('source.html').write_text(driver.page_source)

    # Grab hotel star, svg with find_element is error
    try:
        hotel_star_element = driver.find_element(By.XPATH, '//*[local-name()="svg" and @class="JXZuC d H0"]')
        pattern = r'aria-label="([^"]+)"'
        match = re.search(pattern, hotel_star_element.get_attribute('outerHTML'))
        hotel_star = match.group(1).split(" ")[0]
    except:
        hotel_star = None

    # Grab hotel rating
    
    hotel_rating = driver.find_element(By.XPATH, '//span[@class="uwJeR P"]').text

    try:
        # Grab hotel location score
        loc_score = driver.find_element(By.XPATH, '//span[@class="iVKnd fSVJN"]').text
    except:
        loc_score = 'NULL'
    
    try:
        # Grab hotel restaurant count
        restaurant_count = driver.find_element(By.XPATH, '//span[@class="iVKnd Bznmz"]').text
    except:
        restaurant_count = 'NULL'

    try:
        # Grab hotel attractions
        attractions_count = driver.find_element(By.XPATH, '//span[@class="iVKnd rYxbA"]').text
    except:
        attractions_count = 'NULL'

    # print(hotel_star ,hotel_rating, loc_score, restaurant_count, attractions_count)
    
    # Grab popular mentions
    popular_mentions = []
    popular_mentions_element = driver.find_elements(By.XPATH, '//button[@class="ui_button secondary small JDpQB Md S2 n"]')
    for p in popular_mentions_element:
        p_text = p.text
        popular_mentions.append(p_text)
        
    # Grab hotel style
    hotel_styles = []
    hotel_style_element = driver.find_elements(By.XPATH, '//div[@class="euDRl _R MC S4 _a H"]')
    for h in hotel_style_element:
        hotel_styles.append(h.text)
    hotel_styles = hotel_styles[1:-1]
    
    entry = {
        "Hotel_locID" : loc,
        "Hotel_geoID" : geo,
        "Hotel_star": hotel_star,
        "Hotel_rating": hotel_rating,
        "Location_score": loc_score,
        "Resaurant_count": restaurant_count,
        "Attractions_count": attractions_count,
        "Hotel_styles" : hotel_styles,
        "Popular_mentions": popular_mentions,
    }
    row_data.append(entry)
    return row_data
    

In [7]:
df = pd.DataFrame(columns=["Hotel_locID", "Hotel_geoID", "Hotel_star", "Hotel_rating", "Location_score", "Resaurant_count", "Attractions_count", "Hotel_styles", "Popular_mentions"])


for hotel_url in tqdm(hotels_list):
    try:
        data = get_hotel_info(hotel_url)
        df = pd.concat([df, pd.DataFrame(data)])
    except Exception as e:
        f = Path('error_info_TA_hotel.txt').open('a')
        f.write("Hotel URL: "+ hotel_url + "\n" + "Error info: " + str(e) + "\n")
        f.close()
        # print(e)
        continue
    
f = Path('error_info_TA_hotel.txt').open('a')
ftime = time.strftime("%Y-%m-%d %I:%M:%S %p", time.localtime())
f.write("FINISH!!!!!!!!!! " + ftime + "\n")
f.close()

100%|██████████| 502/502 [32:37<00:00,  3.90s/it]


In [8]:
df

,Hotel_locID,Hotel_geoID,Hotel_star,Hotel_rating,Location_score,Resaurant_count,Attractions_count,Hotel_styles,Popular_mentions
0,119421,32655,3.0,4.0,100,227,58,"[Historic Hotel, Classic]","[downtown la, fitness facilities, athletic fac..."
0,25259222,32655,4.0,5.0,80,38,8,[],"[ac hotel, convention center, front desk, fun ..."
0,84422,32655,4.0,4.5,96,37,29,"[Green, Family]","[the club lounge, lounge staff, city walk, vis..."
0,77866,32655,4.0,4.0,100,187,55,"[Classic, Historic Hotel]","[pershing square, gallery bar, rendezvous cour..."
0,77274,32655,4.0,4.5,100,122,53,"[Historic Hotel, Classic]","[cabana room, spare room, bre smith, tropicana..."
...,...,...,...,...,...,...,...,...,...
0,239812,32655,3.0,3.5,81,48,5,[],"[free parking, universal studios, would stay h..."
0,119421,32655,3.0,4.0,100,227,58,"[Historic Hotel, Classic]","[downtown la, fitness facilities, athletic fac..."
0,17381728,32655,None,3.5,94,82,22,[],"[rooftop bar, amazing rooftop, the van gogh ex..."
0,239812,32655,3.0,3.5,81,48,5,[],"[free parking, universal studios, would stay h..."


In [9]:
# delete duplicated row
df = df.drop_duplicates(subset=['Hotel_locID'])

In [10]:
df

,Hotel_locID,Hotel_geoID,Hotel_star,Hotel_rating,Location_score,Resaurant_count,Attractions_count,Hotel_styles,Popular_mentions
0,119421,32655,3.0,4.0,100,227,58,"[Historic Hotel, Classic]","[downtown la, fitness facilities, athletic fac..."
0,25259222,32655,4.0,5.0,80,38,8,[],"[ac hotel, convention center, front desk, fun ..."
0,84422,32655,4.0,4.5,96,37,29,"[Green, Family]","[the club lounge, lounge staff, city walk, vis..."
0,77866,32655,4.0,4.0,100,187,55,"[Classic, Historic Hotel]","[pershing square, gallery bar, rendezvous cour..."
0,77274,32655,4.0,4.5,100,122,53,"[Historic Hotel, Classic]","[cabana room, spare room, bre smith, tropicana..."
...,...,...,...,...,...,...,...,...,...
0,613619,32655,2.0,2.5,48,119,7,[],[]
0,2505057,32655,1.0,2.5,40,30,1,[Budget],[]
0,575416,32655,2.0,2.0,60,19,4,[],[]
0,84365,32655,2.0,2.5,58,83,4,[],[]


In [11]:
df.to_csv("result_csv/TA_hotels_LA_1002.csv", index=False)